# Dartmouth Implemention of FL - EMBC 2022

Main contributors: Binh Nguyen and Martin Ivanov

In [1]:
#Libraries
import numpy as np
import random
import cv2
import os
import datetime
import pandas as pd
import pywt
import time
import csv
import sys
import matplotlib.pyplot as plt
from math import radians, cos, sin, asin, sqrt
from numpy import *

In [2]:
#Sklearn toolbox libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix 
from sklearn.cluster import KMeans
from skimage.restoration import denoise_wavelet 

In [3]:
#Import utility functions and other functions
from functions import *

# Data Extraction

In [4]:
#Import sensor values ->  (subject, sensor_num)
from functions import sensor_val
# Import EMA values ->  (subject, ema_num)
from functions import ema_val
# Kind of Useless
from functions import activity_ 

In [5]:
# Subject 
s1 =  [('u0'+str(x)) for x in range(9 + 1)]
s2 =  ['u'+str(x) for x in range(10,57)]
subject = s1+s2


In [6]:
# PHQ-9 Analysis

from functions import survey_reader
from functions import survey_encoder_phq

surv_num = 0
phq = survey_reader (surv_num)
dataset_score_pre = []
dataset_score_post = []
user_pre = []
user_post = []

for i in range (len(phq)):
    user = phq.iloc [i,2:-1]
    phq_score = np.sum (survey_encoder_phq(user))
    if (phq.iloc[i,1]=='pre'):
        dataset_score_pre.append(phq_score)
        user_pre.append (phq.iloc[i,0])
    elif (phq.iloc[i,1]=='post'):
        dataset_score_post.append(phq_score)
        user_post.append (phq.iloc[i,0])
        
phq_post = np.vstack((user_post,dataset_score_post))
phq_pre = np.vstack((user_pre,dataset_score_pre))

phq_final = phq_post

In [7]:
print(phq_post)

[['u00' 'u01' 'u02' 'u03' 'u04' 'u05' 'u07' 'u09' 'u10' 'u14' 'u15' 'u16'
  'u17' 'u18' 'u19' 'u20' 'u23' 'u24' 'u27' 'u30' 'u31' 'u32' 'u33' 'u34'
  'u35' 'u36' 'u42' 'u43' 'u44' 'u45' 'u47' 'u49' 'u51' 'u52' 'u53' 'u56'
  'u58' 'u59']
 ['3' '4' '5' '4' '8' '0' '8' '2' '4' '3' '1' '12' '18' '12' '4' '8' '21'
  '7' '7' '0' '5' '2' '25' '6' '7' '1' '0' '4' '2' '2' '1' '8' '0' '15'
  '11' '3' '8' '7']]


# Feature Extraction Set up

In [8]:
# Survey idenfitication 
survey = phq_final

# Create the dataset
from functions import time_difference
from functions import kmeans_cluster
from functions import var_trends_activity
from functions import var_trends_audio
from functions import var_trends_convo
from functions import GPS_csv_reader
from functions import entropy_fct
from functions import wavelet_denoise
from functions import wavelet_denoise2
from functions import percentage_features
from functions import haversine


### Creating empty feature vectors
- Average View 
- Location View
- Trends View

Trends view is composed of Walking, Noise, and Conversation

In [9]:
# Feature vector
avg_view = []
loc_view = []

# Beginning of file Trends
walk_f1 = open("python_walk.txt", "w")
aud_f2 = open("python_noise.txt", "w")
convo_f3 = open("python_convo.txt", "w")


# Feature Extraction

In [10]:
# For loop to run through all subjects
for subject in survey[0]:

    ############################ Average view
    
    # Activity feature 
    sensor_num = 0
    act = sensor_val (subject, sensor_num)
    activity = sensor_val (subject, sensor_num).iloc[:,1]
    
    act_s =  (activity [activity==0])
    act_w =  (activity [activity==1])
    act_r =  (activity [activity==2]) 
    
    # Take length between 2 - 3 seconds (sampling freq)
    r1 = random.uniform(2, 3, len(act_s))
    r2 = random.uniform(2, 3, len(act_w))
    r3 = random.uniform(2, 3, len(act_r))
    
    avg_act_s = np.sum(r1)
    avg_act_w = np.sum(r2)
    avg_act_r = np.sum(r3)

    
    # Conversation 
    sensor_num = 3
    convo = sensor_val (subject, sensor_num)
    x, x, convo_dur = time_difference (convo)
    
    avg_convo_dur = convo_dur
    avg_convo_num = len (convo)


    # Dark features    
    sensor_num = 4
    dark = sensor_val (subject, sensor_num)
    x, x, dark_dur = time_difference (dark)
    
    avg_dark_dur = dark_dur
    avg_dark_num = len (dark)

    
    # Audio features 
    # The total duration when the audio is classified as quiet, noisy and voice in a day.
    sensor_num = 1
    audio = sensor_val (subject, sensor_num)[' audio inference'] 

    aud_s = (audio[audio==0]) 
    aud_v = (audio[audio==1]) 
    aud_n = (audio[audio==2]) 

    # Take length between 2 - 3 seconds (sampling freq)
    r1 = random.uniform(2, 3, len(aud_s))
    r2 = random.uniform(2, 3, len(aud_v))
    r3 = random.uniform(2, 3, len(aud_n))
    
    avg_aud_s = np.sum(r1)
    avg_aud_v = np.sum(r2)
    avg_aud_n = np.sum(r3)
    

    # Phone Lock
    sensor_num = 7
    lock = sensor_val (subject, sensor_num)
    x, x, lock_dur = time_difference (lock)
    
    avg_lock_dur = lock_dur
    avg_lock_num = len (lock)


    hold = avg_act_s, avg_act_w, avg_act_r,\
     avg_convo_dur,avg_convo_num,\
        avg_dark_dur,avg_dark_num,\
            avg_aud_s,avg_aud_v,avg_aud_n,\
                avg_lock_dur, avg_lock_num
    avg_view.append (hold)

    ############################ Trend view 

    # Daily trends in each day for activity
    sensor_num = 0
    act = sensor_val (subject, sensor_num)
    x,x,x,daily_act_s,daily_act_w,daily_act_r = var_trends_activity (act)
    
    # Daily trends in each day for Noise
    sensor_num = 1
    audio = sensor_val (subject, sensor_num)
    x,x,x, daily_aud_s, daily_aud_v, daily_aud_n = var_trends_audio(audio)    
    
    # Daily trends in each day for Conversation
    sensor_num = 3
    convo = sensor_val (subject, sensor_num)
    daily_convo = var_trends_convo (convo)
    
    # Walking, Noise, Conversation to be placed in MATLAB
    # Write to file Daily Activity
    for row in daily_act_w:
        # print (row)
        walk_f1.write (str(row))
        walk_f1.write ('\n')
    walk_f1.write('-1000\n')
    
    # Write to file Daily Noise
    for row in daily_aud_n:
        # print (row)
        aud_f2.write (str(row))
        aud_f2.write ('\n')
    aud_f2.write('-1000\n')
    
    # Write to file Daily Convo
    for row in daily_convo:
        # print (row)
        convo_f3.write (str(row))
        convo_f3.write ('\n')
    convo_f3.write('-1000\n')

    ############################ Location view

    #  GPS feature 
    sensor_num = 5
    gps_df = sensor_val (subject, sensor_num)
    gps_time = GPS_csv_reader(subject,5,'time',1) 
    
    gps_lat = gps_df.iloc [:,3]
    gps_lon = gps_df.iloc [:,4]
    

    # GPS variance is the equation used
    gps_lat_var = np.var (gps_lat)
    gps_lon_var = np.var (gps_lon)
    location_var = (gps_lat_var+gps_lon_var)
    
    
    # Time in location clusters
    # NOTE: these values are basically a percentage of times spent in
    # each clusters. Although data was not collected continuously for all 66
    # days, the variables will not add up to 100%
    tc1,tc2,tc3,tc,n_clust = kmeans_cluster(gps_time,gps_lat,gps_lon, 0)
    
    # Entropy and Normalized entropy
    entropy, norm_entropy = entropy_fct (n_clust,tc)

    # Percentage at home and moving
    home_d, move_p = percentage_features (gps_lat,gps_lon,gps_time,0,subject)
    
    # Total distance using Haversine Formula
    # https://stackoverflow.com/questions/4913349/haversine-formula-in-python-bearing-and-distance-between-two-gps-points
        
    # Filtering techniques involved to avoid large differences in GPS points
    gps_lat_red = gps_lat[gps_lat.between\
                  (gps_lat.quantile(.10),gps_lat.quantile(.90))]
    gps_lon_red = gps_lon[gps_lat.between\
                  (gps_lat.quantile(.10),gps_lat.quantile(.90))]
        
    lon1 = gps_lon_red [0:-1]
    lat1 = gps_lat_red [0:-1]
    lon2 = gps_lon_red [1:]
    lat2 = gps_lat_red [1:]
    
    # V1 - Without filtering out bad points
    # lon1 = gps_lon [0:-1]
    # lat1 = gps_lat [0:-1]
    # lon2 = gps_lon [1:]
    # lat2 = gps_lat [1:]
        
    tot_dist = 0
    for i in range (len(lat1)):
        tot_dist+=haversine(lon1.iloc[i], lat1.iloc[i],\
                            lon2.iloc[i], lat2.iloc[i])

    hold = location_var, tc1,tc2,tc3, entropy, norm_entropy,\
        home_d,move_p,tot_dist
    
    loc_view.append (hold)

F:\Users\enigmen\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:881: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(
F:\Users\enigmen\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:881: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=5.
  warnings.warn(
F:\Users\enigmen\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:881: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=6.
  warnings.warn(
F:\Users\enigmen\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:881: UserWarning: KMeans is known to have a memory leak on Wi

In [11]:
# PHQ-9 Analysis
from functions import survey_reader
from functions import survey_encoder_phq

surv_num = 0
phq = survey_reader (surv_num)
dataset_score_pre = []
dataset_score_post = []
user_pre = []
user_post = []
for i in range (len(phq)):
    user = phq.iloc [i,2:-1]
    phq_score = np.sum (survey_encoder_phq(user))
    if (phq.iloc[i,1]=='pre'):
        dataset_score_pre.append(phq_score)
        user_pre.append (phq.iloc[i,0])
    elif (phq.iloc[i,1]=='post'):
        dataset_score_post.append(phq_score)
        user_post.append (phq.iloc[i,0])
        
phq_post = np.vstack((user_post,dataset_score_post))
phq_pre = np.vstack((user_pre,dataset_score_pre))

phq_final = phq_post



# Label Changes

In [12]:
# Changing label
from sklearn.preprocessing import LabelEncoder
from functions import phq_severity

label_encoder = LabelEncoder()

# phq_lab = label_encoder.fit_transform(phq_severity(\
#     np.asfarray(phq_final[1],float)))

phq_lab = label_encoder.fit_transform(phq_final[1])

phq_label=phq_severity(phq_lab)

# Export and save extracted features to txt files

In [13]:
# PHQ-9
np.savetxt("phq_score.csv", np.array (phq_label), delimiter=', ')  

# Average 
np.savetxt("avg_view.csv", np.array (avg_view), delimiter=', ')  

# Location 
np.savetxt("loc_view.csv", np.array (loc_view), delimiter=', ')  

# Trends
walk_f1.close()
aud_f2.close()
convo_f3.close()